# Trabajo Práctico 1: Análisis Exploratorio de Datos #
## Cientes potenciales a tarjetas de crédito. ##
El [DATASET](https://www.kaggle.com/datasets/sajidhussain3/jobathon-may-2021-credit-card-lead-prediction?select=train.csv) contiene información de clientes de un banco denominado Happy Customer Bank. Es un banco privado de tamaño medio que se ocupa de todo tipo de productos bancarios, como cuentas de ahorro, cuentas corrientes, productos de inversión, productos de crédito, entre otras ofertas.  

En este caso, el Happy Customer Bank desea realizar ventas cruzadas de sus tarjetas de crédito a sus clientes existentes. El banco ha identificado un conjunto de clientes que son elegibles para aceptar estas tarjetas de crédito.

El banco tiene la necesidad, de identificar a los clientes que podrían mostrar intención hacia una tarjeta de crédito recomendada.

<img src = "https://storage.googleapis.com/kaggle-datasets-images/1371711/2277422/2c35585d42b3747882e83e926d0f643e/dataset-cover.png?t=2021-05-27-18-40-42">


 --- 

## 1. Listado de variables y selección 
### a) Variables de Entrada utilizadas

- **Gender:** Género del cliente. (Male/Female). No contiene 'Nans'.



- **Age:** Edad del cliente en años. No contiene 'Nans'.


        
- **Region_Code:** Código string de la Región para los clientes. No contiene 'Nans'. No están detalladas las regiones por nombre, sin embargo al cliente se le puede demostrar la influencia del dato por medio del código.

        
- **Occupation:** Ocupación del cliente. Valores posibles: 'Entrepreneur', 'Other', 'Salaried','Employed'. No contiene 'Nans'.

        
        
- **Channel_Code:** Código para el Canal de Adquisición del Cliente. No contiene 'Nans'. Valores posibles: X1, X2, X3, X4.


        
- **Vintage:** Antiguedad del Cliente en Meses. No contiene 'Nans'.

        
        
- **Credit_Product:** Si el Cliente tiene algún producto de crédito activo como por ejemplo: préstamo de vivienda, préstamo personal, tarjeta de crédito, etc. Valores posibles: True, False. Contiene 'Nans'.


- **Avg_Account_Balance:** Saldo promedio de la cuenta del cliente en los últimos 12 meses. Expresado en int. No contiene 'Nans'.

      
      
- **Is_Active:** Si el cliente estuvo activo en los últimos 3 meses. No contiene 'Nans'.
            No-> El cliente no esta activo | Yes-> el cliente esta activo

### b) Variable de Salida 

- **Is_Lead:**  Si el Cliente está interesado en obtener la tarjeta de Crédito.
            0-> El cliente no esta interesado | 1-> el cliente esta interesado

### c) Variable que no vamos a utilizar 

**ID:** Podría conducir a un sobreajuste.

## 2. Análisis detallado del conjunto de variables

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn as sns
import sklearn
import pytz
import plotly.graph_objects as go

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

# One Hot Encoder
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.pipeline import Pipeline

pd.set_option('display.float_format',lambda x:'%.0f'% x) #Sacar notación científica en pandas
pd.options.display.max_columns = 0

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [ ]:
data_TC = pd.read_csv('train.csv')

BETTER_COLUMN_NAMES = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_TC.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

data_TC.set_index('id', inplace=True)

### a) Balanceo de variable de Salida ###

In [ ]:
import plotly.express as px 
import numpy 
  
random_x = [100, 2000] 
names = ['Esta interesado', 'No esta interesado'] 
  
fig = px.pie(values=data_TC.esta_interesado.value_counts(), names=names) 
fig.show()

Lo que se puede notar es que la variable se encuentra muy desbalanceada, con un 24% de clientes que no estan interesados en adquirir la tarjeta de crédito.
Esto perjudica el entrenamiento, ya que le dificulta generalizar al modelo. 

### b) Comportamiento variables de entrada ###

#### Ocupación

In [ ]:
ocupacion=['Entrepreneur', 'Other', 'Salaried','Employed']

fig = go.Figure(data=[
    go.Bar(name='Esta interesado', x=ocupacion, y=data_TC[data_TC.esta_interesado == 0].ocupacion.value_counts()),
    go.Bar(name='No esta interesado', x=ocupacion, y=data_TC[data_TC.esta_interesado == 1].ocupacion.value_counts()),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Como se puede observar en la gráfica, la mayor cantidad de clientes potenciales tienen como ocupación "Emprendedor".
La dificultad encontrada para el caso de los clientes que son "Empleados" es que la cantidad de casos es muy chica, por lo que se dificulta el dividir los datos y que estos se encuentren representados igualitariamente en Train, Validation y Test.

#### Sexo

In [ ]:
genero=['Masculino', 'Femenino']

fig = go.Figure(data=[
    go.Bar(name='Esta interesado', x=genero, y=data_TC[data_TC.esta_interesado == 0].sexo.value_counts()),
    go.Bar(name='No esta interesado', x=genero, y=data_TC[data_TC.esta_interesado == 1].sexo.value_counts()),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Se puede observar que los datos se encuentran bastante balanceados, aún así, los clientes potenciales masculinos son más.

#### Edad

In [ ]:
df = px.data.tips()
fig = px.histogram(data_TC, x="edad", color="esta_interesado", marginal = 'box', pattern_shape="esta_interesado")
fig.update_layout(bargap=0.1)
fig.show()

En el gráfico podemos observar que la mayor cantidad de clientes potenciales se encuentran entre los 25 y 35 años, de los cuales la mayoría estan interesados en el producto.
Lo que tambien se observa es que los clientes entre 40 y 60 años son los menos interesados, en comparación con los que sí.

#### Antiguedad 

In [ ]:
df = px.data.tips()
fig = px.histogram(data_TC, x="antiguedad", color="esta_interesado", marginal = 'box')
fig.update_layout(bargap=0.1)
fig.show()

La relación que podemos ver, entre la antigüedad de los clientes y si estan interesados o no, es que los no interesados predominan, sin embargo se visualizaron casos que por ejemplo con antigüedad de tres meses hay más interesados que no.

#### Saldo promedio de cuenta

In [ ]:
df = px.data.tips()
fig = px.histogram(data_TC, x="saldo_promedio_cuenta", color="esta_interesado", marginal = 'box')
fig.update_layout(bargap=0.1)
fig.show()

Se puede observar la existencia de muchos extremos en los saldos. 
Tambien se puede ver que la mayor cantidad de los clientes tienen un saldo entre 0 y 4 M. De estos, la cantidad de interesados supera a la de no interesados ampliamente.

#### Código de canal

In [ ]:
canales=['X1','X2','X3','X4']

fig = go.Figure(data=[
    go.Bar(name='Esta interesado', x=canales, y=data_TC[data_TC.esta_interesado == 0].codigo_canal.value_counts()),
    go.Bar(name='No esta interesado', x=canales, y=data_TC[data_TC.esta_interesado == 1].codigo_canal.value_counts()),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

Se puede observar en la gráfica, como predominan en cantidad los registros correspondientes al Canal 'X1', sin embargo, todos comparten la característica de que los interesados superan a los que no. Además en proporción a la cantidad de datos entre los dos canales más utilizados, no existe una brecha grande con respecto a los No interesados. 

### c) Transformaciones

A continuación describimos las transformaciones que se harán a las features disponibles.

- **Gender:** Label Binarizer. Transformamos los valores en 0 y 1, porque es similar a hacer un One-hot encoder de masculino y femenino pero con una sola columna en lugar de 2



- **Age:** Escalar: para que los calores grande sean predominantes sobre valores más chicos de otras columnas. Y Rango: porque no necesitamos saber de que edad en específico, sino que en una franja etaria


        
- **Region_Code:** One-hot encoder, para eliminar las categorías de la columna. 


        
- **Occupation:** One-hot encoder, para eliminar las categorías de la columna. 

        
        
- **Channel_Code:** One-hot encoder, para eliminar las categorías de la columna. 


        
- **Vintage:** Escalar para que los calores grande sean predominantes sobre valores más chicos de otras columnas. Y Rango: porque no necesitamos saber de que antiguedad es el cliente, sino que en una franja etaria

        
        
- **Credit_Product:** Label binarizer. Transformamos los valores en 0 y 1, porque es similar a hacer un One-hot encoder de masculino y femenino pero con una sola columna en lugar de 2. También se eliminan los valores en Null.



- **Avg_Account_Balance:** Escalar para que los calores grande sean predominantes sobre valores más chicos de otras columnas. Y Rango: porque no necesitamos saber cual es el sueldo promedio del cliente, sino que en rango se maneja. Y eliminar valores extremos.

      
      
- **Is_Active:** Label binarizer. Transformamos los valores en 0 y 1, porque es similar a hacer un One-hot encoder de masculino y femenino pero con una sola columna en lugar de 2

Las transformaciones realizadas nos sirven para poder realizar un analisis masestrecho de las variables de entrada. # Falta chamuyar aca, ni idea que poner

### Primer maper

In [ ]:
#esto capaz nos convenga hacerlo en el mapper con sklearn
data_notNull = data_TC[data_TC['tiene_producto_credito_activo'].notna()]

In [ ]:
# Se dividen los datos en train y test
# 60% train, 20% test, 20% validation
#train, not_train = train_test_split(data_notNull, test_size=0.4)
#validation, test = train_test_split(not_train, test_size=0.5)
train, validation = train_test_split(data_notNull, test_size=0.2)

#Dimención (filas, columnas) de los distintos dataset
#print (" Train:", train.shape, '\n', 'Validation:', validation.shape, '\n', 'Test:', test.shape)
print (" Train:", train.shape, '\n', 'Validation:', validation.shape)
#print("first line", "second line", sep="\n")`

In [ ]:
mapper = DataFrameMapper([
    (['sexo'], [LabelBinarizer()]), 
    (['edad'], [StandardScaler()]),
    (['codigo_region'], [OneHotEncoder()]),
    (['ocupacion'], [OneHotEncoder()]),
    (['antiguedad'], [StandardScaler()]),
    (['codigo_canal'], [OneHotEncoder()]),
    (['tiene_producto_credito_activo'], [LabelBinarizer()]),
    (['saldo_promedio_cuenta'], [StandardScaler()]),
    (['es_activo'], [LabelBinarizer()])
], df_out=True) # df_out=True → Es lo que muestra el nombre de la columna

# Lo entrenamos con train
mapper.fit(data_notNull)

In [ ]:
#De ejemplo para ver como funciona
# vemos como transforma un sample:
#sample = data_notNull.sample(5, random_state=20) #Semilla 20

# Sample original:
#sample

In [ ]:
# Sample transformado
mapper.transform(data_notNull)

In [ ]:
# Nombres de los faetures
mapper.transformed_names_

In [ ]:
np.round(mapper.fit_transform(data_notNull), 2)

In [ ]:
np.round(mapper.transform(data_notNull), 1)

In [ ]:
data_TC.edad.rank()

In [ ]:
# Descripción de los datos
data_notNull.describe()

### d) Valores Nulos y Extremos

In [ ]:
# 1. Obtener los nombres de las columas como una lista
col_name = data_TC.columns.tolist()

# 2. Itera sobre la lista de nombres
for column in col_name:
    print ("Valores nulos en <{0}> : {1}".format(column, data_TC[column].isnull().sum()))

In [ ]:
Nulos = data_TC.isnull().sum()
print("Columnas             Cantidad de nulos")
print(Nulos)

In [ ]:
data_TC.count()

Como se puede observar, la feature que describe si el cliente contiene un producto de crédito activo contiene aproximadamente un 12% de variables Null.
La decisión tomada para estos valores fue eliminarlos.

#### Análisis de extremos

In [ ]:
x0 = data_notNull.edad
#x1 = data_notNull.esta_interesado

fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=x0,  name = 'edad'))
#fig.add_trace(go.Box(x=x1))

fig.show()

In [ ]:
x0 = data_notNull.antiguedad                     
#x1 = data_notNull.esta_interesado

fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=x0,  name = 'antiguedad'))
#fig.add_trace(go.Box(x=x1))

fig.show()

In [ ]:
x0 = data_notNull.saldo_promedio_cuenta          
#x1 = data_notNull.esta_interesado

fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=x0,  name = 'saldo promedio cuenta'))
#fig.add_trace(go.Box(x=x1))

fig.show()

In [ ]:
#pd.set_option('display.float_format',lambda x:'%.0f'% x) #Sacar notación científica en pandas
data_notNull.saldo_promedio_cuenta.describe()

In [ ]:
cols = ['saldo_promedio_cuenta'] # one or more

Q1 = data_notNull[cols].quantile(0.25)
Q3 = data_notNull[cols].quantile(0.75)
IQR = Q3 - Q1

data2_notOutLiers = data_notNull[~((data_notNull[cols] < (Q1 - 1.5 * IQR)) |(data_notNull[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
x0 = data2_notOutLiers.saldo_promedio_cuenta          
#x1 = data_notNull.esta_interesado

fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=x0,  name = 'saldo promedio cuenta'))
#fig.add_trace(go.Box(x=x1))

fig.show()

In [ ]:
#pd.set_option('display.float_format',lambda x:'%.0f'% x) #Sacar notación científica en pandas
data2_notOutLiers.saldo_promedio_cuenta.describe()

#### Tipo de cada columna

In [ ]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
# datos.dtypes
data_notNull.info()

### e) Correlación entre las variables

In [ ]:
data_notNull["canal_num"] = pd.to_numeric(data_notNull.codigo_canal.str.replace('X', ''))
data_notNull["region_num"] = pd.to_numeric(data_notNull.codigo_region.str.replace('RG', ''))
data_notNull["producto_credito_num"] = data_notNull.tiene_producto_credito_activo.replace(['No','Yes'], [0, 1])
data_notNull["es_activo_num"] = data_notNull.tiene_producto_credito_activo.replace(['No','Yes'], [0, 1])
data_notNull["ocupacion_num"] = data_notNull.ocupacion.replace(['Entrepreneur', 'Other', 'Salaried','Self_Employed'], [0, 1, 2, 3])

In [ ]:
data_notNull.sample(5)

In [ ]:
#Primero necesita hacer esto para poder dibujar la matriz de correlaciones
corr_matrix = data_notNull.select_dtypes(include=['float64', 'int']).corr(method='pearson')

In [ ]:
# Heatmap matriz de correlaciones
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 20)) # (8, 8) tamaño de la grilla 

sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = True, # Barra del costado
    annot_kws = {"size": 20}, # tamaño nro cuadrilla
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200), # (20, 220, n=200) color de la grilla
    square    = False, 
    ax        = ax
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)

ax.tick_params(labelsize = 10) # (labelsize = 50) tamaño la leyenda de los ejes

Como se puede ver en el gráfico anterior, la variable con mayor correlación a nuestro Target, es la del sueldo promedio de la cuenta.
Esto nos lleva a pensar que personas que generalmente tienen poco saldo, esten dispuestos a obtener una tarjeta de crédito para poder obtener mas bienes o servicios.

## 3. Hipótesis sobre los datos

- Hipótesis 1: 
Los clientes del banco que son empresarios y cuentan con saldo promedio en la cuenta alto, tienen tendencia a presentar interes por una tarjeta de credito.

### Comprobación de hipótesis 1: ###


Para comprobar la hipótesis, en primer lugar vamos a establecer desde qué rango de saldo promedio consideraremos como alto.

Para determinarlo, vamos a obtener el promedio de la columna del saldo promedio.

In [ ]:
mean_df = data_TC['saldo_promedio_cuenta'].mean()
print(mean_df)

Luego de calcular el promedio de la columna saldo_promedio_cuenta, optamos por establecer un saldo promedio 'alto' a un valor superior a 1400000. Vamos agregar una columna "H1" al dataset, la misma devolvera true si el sueldo promedio para ese cliente, supera 1400000 y además es empresario()

In [ ]:
data_TC['H1']= ((data_TC.saldo_promedio_cuenta >= 1400000) & (data_TC.ocupacion=="Entrepreneur"))

In [ ]:
data_TC.sample(10)

En consecuencia, vamos a filtrar con las columnas H1== True y el target: es_activo para evaluar la hipótesis

In [ ]:
data_TC.columns

In [ ]:
ch_df2 = data_TC[data_TC['H1']== True]

In [ ]:
#Obtenemos el promedio interesados, de todos los clientes emprendedores con sueldo en cuenta alto.
mean_df2 = ch_df2['esta_interesado'].mean()
print(mean_df2)

Según el analisis realizado, nuestra hipotesis tiene un 66,5% de valores en que se cumple según el dataset analizado.